<a href="https://colab.research.google.com/github/Donovanphenry/SCAI-Net/blob/main/rainforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### We can construct a mosaic of nearby tiles using this method: https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/36738

### Load data from the shared drive

In [1]:
# mount shared google drive
from google.colab import drive
drive.mount('/gdrive', force_remount=True)
compressed_path = '/gdrive/Shareddrives/CSE 115a/data/compressed'
data_path = '/gdrive/Shareddrives/CSE 115a/data/uncompressed'

Mounted at /gdrive


In [ ]:
# ensure p7zip is installed
!apt install 7z

#### If possible, run the following commands locally:
* consider: https://gist.github.com/jamm1985/df7b8c0abb346d3dbe403bf06ae4f46f

In [ ]:
# unzip data & move it to $data_path -- **WARNING** 7za alone takes an hour to complete!
# !7za x -mmt=128 "$compressed_path"/*.7z -o"$compressed_path"

In [5]:
# locally run this command to un-tar the .tar files, then re-upload them into $data_path
# when running this command in colab, rerun to double check that all files were expanded
!tar --skip-old-files -xvf "$compressed_path"/test-tif-v2.tar -C "$data_path" -i

test-tif-v2/
tar: test-tif-v2: skipping existing file
test-tif-v2/test_28497.tif
^C


### Preprocess data

### Construct model

### Train model

### View results